In [1]:
project = 'The_Dalles'
lookback = 365

In [2]:
# Parameters
project = "The_Dalles"
lookback = 100


In [3]:
import yaml
import pandas as pd
from cwms_read.cwms_read import get_cwms



In [4]:
config = yaml.load(open('config.yml'))
project_dict = config[project]

paths = [v for v in project_dict.values()]
col_names = [k for k in project_dict.keys()]

In [5]:
df = get_cwms(paths, col_names = col_names, public = False, fill = True, lookback = lookback)


In [6]:
df = df.interpolate(limit = 5).dropna()


In [7]:
df = df[df['q_s']>0]

In [8]:
df = df[((df.diff()<df.diff()+df.diff().std()*.2) &(df.diff()>df.diff()-df.diff().std()*1))]
df = df[((df.diff()<df.diff()+df.diff().std()*.2) &(df.diff()>df.diff()-df.diff().std()*1))].dropna()

In [9]:
#convert to SI
df['h_t'] = df['h_t'] * .3058
df['p_atm'] = df['p_atm'] * 133.322
df['q_p'] = df['q_p'] * 0.0283168
df['q_s'] = df['q_s'] * 0.0283168
df['t_water'] = (df['t_water'] - 32) * 5/9

In [10]:
from sklearn.model_selection import train_test_split
x = df
y = pd.DataFrame(df['tdg_tw'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [11]:
data ={}
for df,name in zip([x_train, x_test, y_train, y_test],['x_train', 'x_test', 'y_train', 'y_test']):
    d = {}
    for column in df.columns:
        d.update({column:list(df[column].values)})
    data.update({name:d})
 

In [12]:
"""{}.ipynb""".format(project)
import papermill as pm

pm.record(project, data)